In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from movie_metadata_table import MovieMetadataTable

import sys
sys.path.append("../algorithms")
from deepfm import DeepFM # type: ignore

In [3]:
movie_metadata_table = MovieMetadataTable(
    movie_ids_file="../data/movie_ids.json",
    movie_data_vectorized_file="../data/vectorizing/movie_data_vectorized.csv",
    nlp_vectors_file="../data/vectorizing/nlp_vectors.pt",
)

In [29]:
import json
import random

random.seed(8888)

# get movie ids, clipping off movie ids that are not in the movie tensor
all_movie_ids = json.load(open("../data/movie_ids.json"))[:movie_metadata_table.movie_tensor.shape[0]]
random.shuffle(all_movie_ids)

ratings = pd.read_csv("../data/ratings_export.csv")

train_user_ids = None
train_movie_ids = all_movie_ids[:30]

if train_user_ids is not None:
    ratings = ratings[ratings['user_id'].isin(train_user_ids)]

if train_movie_ids is not None:
    ratings = ratings[ratings['movie_id'].isin(train_movie_ids)]

all_user_ids = ratings['user_id'].unique()
all_movie_slugs = ratings['movie_id'].unique()

device = "cuda" if torch.cuda.is_available() else "cpu"
user_vector_size = 64
user_id_to_index = {user_id: i for i, user_id in enumerate(all_user_ids)}

In [19]:
print(len(movie_metadata_table.movie_ids), movie_metadata_table.movie_tensor.shape)

285961 torch.Size([270422, 231])


In [52]:
user_embedding_table = nn.Embedding(len(all_user_ids), user_vector_size).to(device)
deepfm = DeepFM(
    movie_metadata_table.movie_vector_size,
    user_vector_size,
    num_dense_movie_embeddings=8,
    num_dense_user_embeddings=4,
    dense_embedding_size=16,
    mlp_sizes=[16, 16, 1],
).to(device)

In [53]:
import tqdm

# note: if we want to get a massive speedup,
# we can probably use a sparse optimization scheme somehow
optim = torch.optim.Adam([
    *deepfm.parameters(),
    *user_embedding_table.parameters()
], lr=0.001)

# determine how to give a reward
# we will just give a reward if the user rated the movie >= 7/10

loss_type = "mse"
reward_rating_cutoff = 7

avg_reward_per_movie_numer = torch.zeros(len(train_movie_ids), device=device)
avg_reward_per_movie_denom = torch.zeros(len(train_movie_ids), device=device)

# iterate over ratings in random batches
indexes = torch.randperm(len(ratings))
batch_size = 16

for epoch in range(100):
    epoch_loss_total = 0
    epoch_corrects = 0
    epoch_correct_baseline = 0
    epoch_seen = 0
    for batch_start in (pbar := tqdm.tqdm(range(0, len(indexes), batch_size))):
        batch = ratings.iloc[indexes[batch_start:batch_start + batch_size]]

        movie_slugs = [str(x) for x in batch['movie_id'].values]
        train_user_ids = batch['user_id'].values
        ratings_ = batch['rating_val'].values

        user_indices = torch.tensor([user_id_to_index[user_id] for user_id in train_user_ids], device=device)
        user_vectors = user_embedding_table(user_indices)
        movie_vectors = movie_metadata_table(movie_slugs).to(device)
        movie_average_ratings = movie_vectors[:, 2].to(device)

        predictions = deepfm(movie_vectors.float(), user_vectors.float()).squeeze(-1)
        rewards = torch.tensor(ratings_ >= reward_rating_cutoff, device=device, dtype=torch.float32)

        assert not torch.any(rewards.isnan())

        if loss_type == 'mse':
            # resembles learning q function
            loss = F.mse_loss(predictions, rewards)
        elif loss_type == 'binary_crossentropy':
            # loosely resembles policy gradient
            loss = F.binary_cross_entropy_with_logits(predictions, rewards.float())

        optim.zero_grad()
        loss.backward()
        optim.step()
        
        epoch_corrects += ((predictions >= 0.5) == rewards).sum()
        
        movie_pos_in_tracking_vector = [train_movie_ids.index(slug) for slug in movie_slugs]
        avg_reward_per_movie_numer[movie_pos_in_tracking_vector] += rewards
        avg_reward_per_movie_denom[movie_pos_in_tracking_vector] += 1
        baseline_reward_predictions = (avg_reward_per_movie_numer/avg_reward_per_movie_denom)[movie_pos_in_tracking_vector]
        baseline_corrects = (baseline_reward_predictions >= 0.5) == rewards
        epoch_correct_baseline += baseline_corrects.sum()
        epoch_loss_total += loss.item() * len(user_indices)
        epoch_seen += len(user_indices)

        pbar.set_postfix(loss=epoch_loss_total / epoch_seen, accuracy=(epoch_corrects / epoch_seen).item(), baseline=(epoch_correct_baseline / epoch_seen).item())


100%|██████████| 96/96 [00:00<00:00, 209.15it/s, accuracy=0.81, baseline=0.711, loss=0.22]  
